In [30]:
import csv
import json
import os
import random

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer as CV
import nltk
import kenlm

import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
sfp_data = pd.read_csv("../sample_data/sfp_data_v4.csv")

In [3]:
td_data = pd.read_csv("../sample_data/td_data_v4.csv")

In [5]:
sfp_authors = sfp_data['author'].unique()
td_authors = td_data['author'].unique()

sfp_posts = sfp_data.loc[sfp_data['subreddit'] == 'SandersForPresident']
td_posts = td_data.loc[td_data['subreddit'] == 'The_Donald']

sfp_min_times = [min(sfp_posts.loc[sfp_posts['author'] == auth]['created_utc']) for auth in sfp_authors]
td_min_times = [min(td_posts.loc[td_posts['author'] == auth]['created_utc']) for auth in td_authors]

In [7]:
print(sfp_data.shape)
print(td_data.shape)
print(len(sfp_authors))
print(len(td_authors))
print(sfp_posts.shape)
print(td_posts.shape)

(4868749, 5)
(4003940, 5)
2000
2000
(78037, 5)
(55521, 5)


In [6]:
sfp_politics = sfp_data.loc[sfp_data['subreddit'] == 'politics']
td_politics = td_data.loc[td_data['subreddit'] == 'politics']
print(sfp_politics.shape)
print(td_politics.shape)

(233708, 5)
(89894, 5)


In [8]:
sfp_politics_before = pd.DataFrame([], columns = sfp_politics.columns)
sfp_politics_after = pd.DataFrame([], columns = sfp_politics.columns)

for i in range(len(sfp_authors)):
    curr_auth_posts = sfp_politics.loc[sfp_politics['author'] == sfp_authors[i]]
    
    if len(curr_auth_posts) > 0:
        sfp_politics_before = \
sfp_politics_before.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] < sfp_min_times[i]])
        
        sfp_politics_after = \
sfp_politics_after.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] > sfp_min_times[i]])
    

In [9]:
td_politics_before = pd.DataFrame([], columns = td_politics.columns)
td_politics_after = pd.DataFrame([], columns = td_politics.columns)

for i in range(len(td_authors)):
    curr_auth_posts = td_politics.loc[td_politics['author'] == td_authors[i]]
    
    if len(curr_auth_posts) > 0:
        td_politics_before = \
td_politics_before.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] < td_min_times[i]])
        
        td_politics_after = \
td_politics_after.append(curr_auth_posts.loc[curr_auth_posts['created_utc'] > td_min_times[i]])
    

In [10]:
sfp_politics_before_text = sfp_politics_before.loc[sfp_politics_before['body'].notna()]['body'].values
sfp_politics_after_text = sfp_politics_after.loc[sfp_politics_after['body'].notna()]['body'].values

td_politics_before_text = td_politics_before.loc[td_politics_before['body'].notna()]['body'].values
td_politics_after_text = td_politics_after.loc[td_politics_after['body'].notna()]['body'].values

sfp_posts_text = sfp_posts.loc[sfp_posts['body'].notna()]['body'].values
td_posts_text = td_posts.loc[td_posts['body'].notna()]['body'].values

In [14]:
text_arrs = [sfp_politics_before_text,
             sfp_politics_after_text,
             td_politics_before_text,
             td_politics_after_text,
             sfp_posts_text,
             td_posts_text]

test_arrs = []

for i in range(len(text_arrs)):
    lm_test_arr = \
[(' '.join(nltk.word_tokenize(s)).lower()) for v in text_arrs[i] for s in nltk.sent_tokenize(v)]
    test_arrs.append(lm_test_arr)

In [19]:
for i in range(len(test_arrs)):
    print(len(test_arrs[i]))
    print(test_arrs[i][:10])
    print("\n\n")

367959
["presidential run aside , i do n't understand why you make the assumption that he 's nothing without his comedy writers .", 'he is one of his own comedy writers .', 'aside from being the host of the show , colbert is an executive producer and has written about as many episodes as any other writer .', '[ imdb source ] ( http : //www.imdb.com/title/tt0458254/fullcredits )', "that 's because those numbers include [ superdelegates ] ( http : //en.wikipedia.org/wiki/superdelegate ) , which are not won via the caucus .", 'obama won more pledged delegates in iowa , which are assigned based on the iowan caucus .', "but questionmark 's original comment stated that hillary won more delegates in iowa , which is not true .", "she won fewer pledged delegates than obama , and even when you include superdelegates , she 's not ahead in iowa .", 'i mentioned that in my [ previous comment ] ( http : //politics.reddit.com/info/65zyd/comments/c02xwsj ) .', 'the numbers you posted represent the nat

In [56]:
for i in range(len(test_arrs)):
    print(len(test_arrs[i]))

367959
353333
216725
60428
204878
115589


In [24]:
filenames = ['politics_train.arpa',
             'sfp_politics_before.arpa', 
             'sfp_politics_after.arpa',
             'td_politics_before.arpa',
             'td_politics_after.arpa',
             'sfp_posts.arpa',
             'td_posts.arpa']

In [25]:
language_models = []

for i in range(len(filenames)):
    curr_lm = os.path.join(os.path.dirname('Language models.ipynb'), '..', 'language_models', filenames[i])
    curr_model = kenlm.LanguageModel(curr_lm)
    print('{0}-gram model'.format(curr_model.order))
    language_models.append(curr_model)

2-gram model
2-gram model
2-gram model
2-gram model
2-gram model
2-gram model
2-gram model


In [38]:
int(2.7)

2

In [67]:
np.random.choice(['a','b','c','d'], 4, replace=False)

array(['a', 'b', 'c', 'd'], dtype='<U1')

In [68]:
def bootstrap(text1, text2, scoring_func, num_iters = 1000, sample_size = 1000):
    lower_size = min(len(text1), len(text2))
    text1_new = np.random.choice(text1, lower_size, replace=False)
    text2_new = np.random.choice(text2, lower_size, replace=False)
    
    scores_all = [scoring_func(s) for s in (text1 + text2)]
    bootstrap1 = []
    bootstrap2 = []
    random.seed(10)
    
    for i in range(num_iters):
        scores1 = random.sample(scores_all, sample_size)
        scores2 = random.sample(scores_all, sample_size)
        
        bootstrap1.append(sum(scores1)/sample_size)
        bootstrap2.append(sum(scores2)/sample_size)
        
    return (bootstrap1, bootstrap2)  

In [69]:
def calculate_values(text1, text2, scoring_func, num_iters = 1000, sample_size = 1000):
    random.seed(10)
    text1_sample = random.sample(text1, sample_size)
    text2_sample = random.sample(text2, sample_size)
    scores1 = [scoring_func(s) for s in text1_sample]
    scores2 = [scoring_func(s) for s in text2_sample]

    mean1 = sum(scores1)/len(scores1)
    mean2 = sum(scores2)/len(scores2)
    print(mean1)
    print(mean2)
    print()
    
    bootstrap1, bootstrap2 = bootstrap(text1, text2, scoring_func, num_iters)
    model_diff = mean1 - mean2
    print(model_diff)
    print()
    
    bootstrap_diffs = [bootstrap1[i] - bootstrap2[i] for i in range(len(bootstrap1))]
    print(sum([i > model_diff for i in bootstrap_diffs])/len(bootstrap_diffs))

In [70]:
# sfp_politics_before, td_politics_before, sfp_politics_before
calculate_values(test_arrs[0], test_arrs[2], language_models[1].perplexity, num_iters=1000, sample_size = 20000)

466.0915190667158
464.0644101356655

2.027108931050293

0.512


In [71]:
# sfp_politics_before, td_politics_before, td_politics_before
calculate_values(test_arrs[0], test_arrs[2], language_models[3].perplexity, num_iters=1000, sample_size = 20000)

415.26142824930616
390.3400073401792

24.921420909126937

0.438


In [72]:
# sfp_politics_after, td_politics_after, sfp_politics_after
calculate_values(test_arrs[1], test_arrs[3], language_models[2].perplexity, num_iters=1000, sample_size = 20000)

348.7865394483283
516.1244845746966

-167.33794512636825

0.877


In [73]:
# sfp_politics_after, td_politics_after, td_politics_after
calculate_values(test_arrs[1], test_arrs[3], language_models[4].perplexity, num_iters=1000, sample_size = 20000)

352.5069705611644
414.18445782481314

-61.67748726364874

0.82


In [74]:
# sfp_politics_before, td_politics_before, sfp_posts
calculate_values(test_arrs[0], test_arrs[2], language_models[5].perplexity, num_iters=1000, sample_size = 20000)

452.27702955300754
442.24635122921

10.030678323797531

0.478


In [75]:
# sfp_politics_before, td_politics_before, td_posts
calculate_values(test_arrs[0], test_arrs[2], language_models[6].perplexity, num_iters=1000, sample_size = 20000)

411.0995839632289
390.2742375488254

20.825346414403498

0.376


In [76]:
# sfp_politics_after, td_politics_after, sfp_posts
calculate_values(test_arrs[1], test_arrs[3], language_models[5].perplexity, num_iters=1000, sample_size = 20000)

361.5910337341523
526.1723508792212

-164.5813171450689

0.92


In [77]:
# sfp_politics_after, td_politics_after, td_posts
calculate_values(test_arrs[1], test_arrs[3], language_models[6].perplexity, num_iters=1000, sample_size = 20000)

357.7513617089054
376.5504233294564

-18.799061620551015

0.607


In [78]:
# sfp_politics_before, td_politics_before, politics
calculate_values(test_arrs[0], test_arrs[2], language_models[0].perplexity, num_iters=1000, sample_size = 20000)

386.99331096744527
395.1014610810287

-8.108150113583406

0.531


In [79]:
# sfp_politics_after, td_politics_after, politics
calculate_values(test_arrs[1], test_arrs[3], language_models[0].perplexity, num_iters=1000, sample_size = 20000)

349.8563032214515
439.71758078008185

-89.86127755863032

0.93


In [80]:
# sfp_posts, td_posts, sfp_posts
calculate_values(test_arrs[4], test_arrs[5], language_models[5].perplexity, num_iters=1000, sample_size = 20000)

372.3425881771748
1437.7450103397887

-1065.402422162614

0.998


In [81]:
# sfp_posts, td_posts, td_posts
calculate_values(test_arrs[4], test_arrs[5], language_models[6].perplexity, num_iters=1000, sample_size = 20000)

383.91765532531605
658.0044753733365

-274.08682004802046

0.963


In [82]:
# td_posts, sfp_posts, td_posts
calculate_values(test_arrs[5], test_arrs[4], language_models[6].perplexity, num_iters=1000, sample_size = 20000)

654.1731912362627
437.5470999445293

216.62609129173342

0.069


In [83]:
# sfp_posts, td_posts, politics
calculate_values(test_arrs[4], test_arrs[5], language_models[0].perplexity, num_iters=1000, sample_size = 20000)

393.96497182012
987.3871161287846

-593.4221443086645

0.999


In [84]:
# sfp_politics_before, sfp_politics_after, politics
calculate_values(test_arrs[0], test_arrs[1], language_models[0].perplexity, num_iters=1000, sample_size = 20000)

386.99331096744527
357.0470253137992

29.946285653646044

0.321


In [85]:
# td_politics_before, td_politics_after, politics
calculate_values(test_arrs[2], test_arrs[3], language_models[0].perplexity, num_iters=1000, sample_size = 20000)

380.8497134050213
452.1804452783587

-71.33073187333741

0.849


## Statistical analyses